## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,51.46,35,40,24.16,light rain
1,1,New Norfolk,AU,-42.7826,147.0587,43.75,72,87,3.00,overcast clouds
2,2,Bengkulu,ID,-3.8004,102.2655,75.54,90,91,3.42,overcast clouds
3,3,Lardos,GR,36.0942,28.0157,75.94,68,0,7.23,clear sky
4,4,Cabo San Lucas,MX,22.8909,-109.9124,96.62,63,40,4.00,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bengkulu,ID,-3.8004,102.2655,75.54,90,91,3.42,overcast clouds
3,3,Lardos,GR,36.0942,28.0157,75.94,68,0,7.23,clear sky
5,5,Bubaque,GW,11.2833,-15.8333,82.72,77,100,9.82,overcast clouds
10,10,Hilo,US,19.7297,-155.0900,79.29,91,90,5.75,moderate rain
13,13,Faanui,PF,-16.4833,-151.7500,79.14,76,41,13.27,scattered clouds
16,16,Sabha,LY,27.0377,14.4283,78.49,13,0,12.06,clear sky
18,18,Vaini,TO,-21.2000,-175.2000,75.36,88,75,4.61,broken clouds
19,19,Buala,SB,-8.1450,159.5921,81.30,80,96,5.14,overcast clouds
23,23,Banjar,ID,-8.1900,114.9675,75.06,86,68,6.73,broken clouds
24,24,Upala,CR,10.8979,-85.0145,90.00,77,45,3.56,scattered clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,75.54,overcast clouds,-3.8004,102.2655,
3,Lardos,GR,75.94,clear sky,36.0942,28.0157,
5,Bubaque,GW,82.72,overcast clouds,11.2833,-15.8333,
10,Hilo,US,79.29,moderate rain,19.7297,-155.0900,
13,Faanui,PF,79.14,scattered clouds,-16.4833,-151.7500,
16,Sabha,LY,78.49,clear sky,27.0377,14.4283,
18,Vaini,TO,75.36,broken clouds,-21.2000,-175.2000,
19,Buala,SB,81.30,overcast clouds,-8.1450,159.5921,
23,Banjar,ID,75.06,broken clouds,-8.1900,114.9675,
24,Upala,CR,90.00,scattered clouds,10.8979,-85.0145,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        
        print("Hotel not found... skipping.")
print("End of search")      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
End of search


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,75.54,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
3,Lardos,GR,75.94,clear sky,36.0942,28.0157,Lindos Princess Beach Hotel
5,Bubaque,GW,82.72,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
10,Hilo,US,79.29,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Faanui,PF,79.14,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
...,...,...,...,...,...,...,...
703,Lubao,PH,81.14,overcast clouds,14.9167,120.5500,Hacienda Gracia
705,Limbang,MY,80.91,overcast clouds,4.7500,115.0000,Purnama Hotel
707,Conde,BR,80.06,few clouds,-7.2597,-34.9075,Pousada Beija Flor
708,Port Blair,IN,80.73,overcast clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))